# Extracting distance between amino acids, contacts and attentions to train and test ML classifiers

## Import .py file

In [1]:
# Set path to directory containing .py files
# Import .py file to extract distance between amino acids, contacts and attentions
# All necessary packages are imported within the .py file

import sys
sys.path.append("../py")

# ESM-2 model for protein embeddings is esm.pretrained.esm2_t33_650M_UR50D()
from extract_contact_aa_distance_attentions import * 


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [2]:
# Ignore warnings when importing PDB structure files
warnings.simplefilter('ignore', PDBConstructionWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Set variable for parsing PDB structural files
parser = PDBParser()  

## Extract Sequences from CASP7 Dataset

In [3]:
## Make sure to update directory paths from .py file prior to executing code

# Parse CASP training_95 file, extract all sequence IDs and sequences from dataset
# CASP training_95 file contains seed sequences for sequence clusters at 95% similarity

prot_data_dict = parse_casp7_file(casp_95)

for casp_id, casp_seq in list(prot_data_dict.items())[:3]:
    print(casp_id)
    print(casp_seq)
    print()

1RSO_2_B
GLLAAERAVSQVLDSLEEIHALTDSSEKDLDFLHSVFQDQHLHTLLDLYDKINTKS

3PCG_1_A
PIELLPETPSQTAGPYVHIGLALEAAGNPTRDQEIWNRLAKPDAPGEHILLLGQVYDGNGHLVRDSFLEVWQADANGEYQDAYNLENAFNSFGRTATTFDAGEWTLHTVKPGVVNNAAGVPMAPHINISLFARGINIHLHTRLYFDDEAQANAKCPVLNLIEQPQRRETLIAKRCEVDGKTAYRFDIRIQGEGETVFFDF

1YBD_1_A
MTQQIKYKRVLLKLSGESLMGSDPFGINHDTIVQTVGEIAEVVKMGVQVGIVVGGGNIFRGVSAQAGSMDRATADYMGMMATVMNALALKDAFETLGIKARVQSALSMQQIAETYARPKAIQYLEEGKVVIFAAGTGNPFFTTDTAAALRGAEMNCDVMLKATNVDGVYTADPKKDPSATRYETITFDEALLKNLKVMDATAFALCRERKLNIVVFGIAKEGSLKRVITGEDEGTLVHC



In [4]:
# CASP datasets contains sequence IDs that correspond to different chains in a sequence
# I.E. 1RSO_2_B refers to the 2nd part of chain B of the protein 1RSO

## To be sure I extract the CASP sequence that corresponds to the sequence in the 
# PDB structural file, I extract CASP IDs that occur once
# Which means that the stored sequence is the entire sequence for the protein
# This is because some sequences in the dataset only have a label for 1 chain

# Calculate occurence of sequence IDs
protein_id_counts = Counter(protein_id.split('_')[0] for protein_id in prot_data_dict)

# Store IDs that only occur once as single_occurence_ids
single_occurence_ids = [protein_id for protein_id, count in protein_id_counts.items() if count == 1]

print('Number of Unique IDs: ', len(single_occurence_ids), '\n')
print(single_occurence_ids[:5])

Number of Unique IDs:  8494 

['1YBD', '1PSM', '1DM0', '1WFO', '1YOA']


In [16]:
'1R4Q' in prot_data_dict.keys()

False

## Extract FASTA files from PDB database from identified CASP Protein IDs

In [5]:
## Take the IDs that occur once in the CASP training_95 dataset and extract the sequences
# from the PDB database. These sequences are used in generating PDB structural data.
# I.E. calculating arnstrong distance.

# for pdb_id in single_occurence_ids:
#     download_fasta(pdb_id, fasta_dir, downloadurl="https://www.rcsb.org/fasta/entry/")

# Load sequence ID and sequences from PDB into a dictionary
protein_data = load_fastas(fasta_dir)

print('Number of Sequences: ', len(protein_data), '\n')

for pdb_id, pdb_seq in list(protein_data.items())[:3]:
    print(pdb_id)
    print(pdb_seq)
    print()

Number of Sequences:  2682 

1C4R
GHAGTTYIFSKGGGQITYKWPPNDRPSTRADRLAIGFSTVQKEAVLVRVDSSSGLGDYLELHIHQGKIGVKFNVGTDDIAIEESNAIINDGKYHVVRFTRSGGNATLQVDSWPVIERYPAGRQLTIFNSQATIIIGGKEQGQPFQGQLSGLYYNGLKVLNMAAENDANIAIVGNVRLVGEVP

1UE8
MYDWFKQMRKESPVYYDGKVWNLFKYEDCKMVLNDHKRFSSNLTGYNDKLEMLRSGKVFFDIPTRYTMLTSDPPLHDELRNLTADAFNPSNLPVDFVREVTVKLLSELDEEFDVIESFAIPLPILVISKMLGINPDVKKVKDWSDLVALRLGRADEIFSIGRKYLELISFSKKELDSRKGKEIVDLTGKIANSNLSELEKEGYFILLMIAGNETTTNLIGNAIEDFTLYNSWDYVREKGALKAVEEALRFSPPVMRTIRVTKEKVKIRDQVIDEGELVRVWIASANRDEEVFKDPDSFIPDRTPNPHLSFGSGIHLCLGAPLARLEARIALEEFAKKFRVKEIVKKEKIDNEVLNGYRKLVVRVERT

1I24
MRGSHHHHHHGSRVMVIGGDGYCGWATALHLSKKNYEVCIVDNLVRRLFDHQLGLESLTPIASIHDRISRWKALTGKSIELYVGDICDFEFLAESFKSFEPDSVVHFGEQRSAPYSMIDRSRAVYTQHNNVIGTLNVLFAIKEFGEECHLVKLGTMGEYGTPNIDIEEGYITITHNGRTDTLPYPKQASSFYHLSKVHDSHNIAFTCKAWGIRATDLNQGVVYGVKTDETEMHEELRNRLDYDAVFGTALNRFCVQAAVGHPLTVYGKGGQTRGYLDIRDTVQCVEIAIANPAKAGEFRVFNQFTEQFSVNELASLVTKAGSKLGLDVKKMTVPNPRVEAEEHYYNAKHTKLMELGLEPHYLSDSLLDSLLNFAVQFKDRVDTKQIMPSVSWKKIGVKTKSMT

## Extract sequences that are the same in PDB database and CASP dataset

In [6]:
# Store sequence IDs that have sequences that are the same in both PDB database and
# CASP training_95 dataset
# Double checking if the sequences are uniform is important to ensure we are comparing
# the same proteins and chains that are the seed sequences in the CASP dataset clusters

same_sequence_ids = check_casp_pdb_seqs(protein_data)

print('Number of Uniform Sequences: ', len(same_sequence_ids), '\n')

Number of Uniform Sequences:  1215 



## Extract PDB structural data

In [ ]:
## Takes pdb_ids that have been extracted above and downloads the pdb structural file from
# the pdb database.
# These files will be used when generating contact sites.


# for pdb_id in same_sequence_ids:
#     download_pdb(pdb_id, structure_dir, downloadurl="http://files.rcsb.org/download/")



## Calculate distance between amino acids in the sequence and arnstrong distance between amino acids from PDB structural file

In [7]:
## To make sure we have a balanced number of contacts and non-contacts for each protein
# We make a subset of non-contact sites that are randomly selected and
# has the same number of amino acid pairs as in-contact sites

in_contact_sites, non_contact_sites, subset_non_contact_sites = contacts_per_pdb(same_sequence_ids,protein_data)

# Assuming in_contact_sites and subset_non_contact_sites are dictionaries
for sequence_id_in_contact, sequence_id_non_contact in zip(list(in_contact_sites.keys())[:1], list(subset_non_contact_sites.keys())[:1]):
    # If you want to process only 'in_contact_sites'
    for instance in random.sample(in_contact_sites[sequence_id_in_contact], 3):
        print(instance)

for instance in random.sample(subset_non_contact_sites[sequence_id_non_contact], 3):
    print(instance)


{'res_1': 215, 'res_2': 212, 'sig_1': 'T', 'sig_2': 'N', 'aa_dist': 3, 'arn_dist': 4.8835087, 'in_contact': True}
{'res_1': 274, 'res_2': 264, 'sig_1': 'E', 'sig_2': 'K', 'aa_dist': 10, 'arn_dist': 4.846522, 'in_contact': True}
{'res_1': 282, 'res_2': 256, 'sig_1': 'I', 'sig_2': 'R', 'aa_dist': 26, 'arn_dist': 4.9069557, 'in_contact': True}
{'res_1': 155, 'res_2': 54, 'sig_1': 'D', 'sig_2': 'R', 'aa_dist': 101, 'arn_dist': 26.104069, 'in_contact': False}
{'res_1': 148, 'res_2': 120, 'sig_1': 'A', 'sig_2': 'I', 'aa_dist': 28, 'arn_dist': 13.640676, 'in_contact': False}
{'res_1': 325, 'res_2': 300, 'sig_1': 'L', 'sig_2': 'P', 'aa_dist': 25, 'arn_dist': 16.530783, 'in_contact': False}


In [11]:
# Store in_contact dictionary and subset of non_contact dictionary as a single dictionary
# which will be used in ML train/test split

# Some sequences chains that are found to have no contacts are removed from the dictionary

contact_data = generate_contact_data(in_contact_sites, subset_non_contact_sites)

print('Number of sequences: ', len(contact_data))

Number of sequences:  1102


## Collect X, y data for ML model training

In [12]:
# Randomly select sequences from same_sequence_ids to reduce computational demand for training
seed_value = 67
random.seed(seed_value)
n_sequences = 300

sequence_ids = random.sample(same_sequence_ids, n_sequences)
# sequence_ids = same_sequence_ids
print('n_sequences: ', n_sequences)
print(sequence_ids[:5])

n_sequences:  300
['1BUE', '1FK5', '1QK9', '1PPQ', '1T5Q']


In [13]:
# The following function takes sequence IDs, generates ESM-2 embeddings, takes instances
# from contact_data, extracts combinations of residue 1 and residue 2, extracts attentions
# from all of the layers and heads for these combinations and extracts whether the residues 
# are in contact or not from earlier arnstrong distance calculations. 

# Attention vectors are stored as X, and residue contact values (True/False) are stored as Y
# Iterations represent how many sequences have been iterated through in the function
# Iterations = n_sequences

X, y = output_x_y(sequence_ids, contact_data, protein_data)

Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Itera

In [14]:
# Train/Test split
# Previous experiment suggests test_size = 0.4 is optimal for best performance
# Due to subseting non-contact instances to be = to in-contact instances
# number of positive cases == number of negative cases

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.4, random_state=42)

print('Number of Amino Acid Instances: ', len(X_train))
print('Number of positive instances: ', round(len(X_train)/2))

Number of Amino Acid Instances:  9699
Number of positive instances:  4850


In [19]:
(X_train[0])

[tensor(0.0183),
 tensor(0.0051),
 tensor(0.0120),
 tensor(0.0068),
 tensor(0.0118),
 tensor(0.0155),
 tensor(0.0064),
 tensor(0.0090),
 tensor(0.0034),
 tensor(0.0173),
 tensor(0.0148),
 tensor(0.0172),
 tensor(0.0117),
 tensor(0.0131),
 tensor(0.0041),
 tensor(0.0080),
 tensor(0.0159),
 tensor(0.0095),
 tensor(0.0098),
 tensor(0.0022),
 tensor(4.2884e-05),
 tensor(3.5786e-07),
 tensor(0.0056),
 tensor(0.0048),
 tensor(0.0065),
 tensor(0.0187),
 tensor(0.0088),
 tensor(3.2372e-08),
 tensor(2.3364e-09),
 tensor(0.0060),
 tensor(0.0048),
 tensor(0.0418),
 tensor(0.0033),
 tensor(0.0052),
 tensor(0.0041),
 tensor(0.0001),
 tensor(4.3949e-08),
 tensor(0.0312),
 tensor(0.1279),
 tensor(0.0102),
 tensor(0.0062),
 tensor(0.0021),
 tensor(0.0010),
 tensor(0.0174),
 tensor(0.0002),
 tensor(0.1701),
 tensor(0.0071),
 tensor(0.0150),
 tensor(0.0113),
 tensor(0.0110),
 tensor(0.0267),
 tensor(0.0010),
 tensor(0.0045),
 tensor(0.0032),
 tensor(0.0035),
 tensor(0.0107),
 tensor(0.0015),
 tensor(1.5

## Train Models

In [98]:
# Model hyperparameters optimized using Grid-Search CV on 300 randomly selected sequences
# 9,699 amino acid instances

# Train Random Forest Classifier

rf_clf = RandomForestClassifier(criterion='gini', max_depth=8,max_features='sqrt',n_estimators = 1000,random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, n_estimators=1000, random_state=42)

In [17]:
# Model hyperparameters optimized using Grid-Search CV on 300 randomly selected sequences
# 9,699 amino acid instances

# Train Linear SVC

linear_svc = LinearSVC(C=10, class_weight='balanced', dual=True, loss='hinge',max_iter=200000)
linear_svc.fit(X_train, y_train)

LinearSVC(C=10, class_weight='balanced', dual=True, loss='hinge',
          max_iter=200000)

In [19]:
# Model hyperparameters optimized using Grid-Search CV on 300 randomly selected sequences
# 9,699 amino acid instances

# Train Logistic Regression

lr_clf = LogisticRegression(C=100, max_iter=200000, solver='saga')
lr_clf.fit(X_train, y_train)

LogisticRegression(C=100, max_iter=200000, solver='saga')

## Test a randomly generated sequence against trained models

In [83]:
# List of standard amino acids (excluding stop codons)
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 
               'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

# Generate a random sequence of 300 amino acids
random_seq = ''.join(random.choices(amino_acids, k=300))

rand_protein_data = {}
rand_protein_data['random1'] = random_seq

In [84]:
# Get random sequence embedding

random_embedding = generate_embeddings('random1',rand_protein_data)

In [91]:
# Get random sequence embeddings

random_x = []

for res_1 in random.sample(range(300),80):
    for res_2 in random.sample(range(300),80):
        if res_1==res_2:
            continue
        else:
#             print(res_1,res_2)
            random_x.append(get_x_y(random_embedding, res_1, res_2))

In [92]:
# Take a subset of amino acid pair instances
n = 5000
random_x_test = random.sample(random_x, n)

In [99]:
# Make predictions

rf_rand_y_pred = rf_clf.predict(random_x_test)
# lr_rand_y_pred = lr_clf.predict(random_x_test)
# svc_rand_y_pred = linear_svc.predict(random_x_test)

In [20]:
rf_false_count = 0
rf_true_count = 0

for i in rf_rand_y_pred:
    if i == False:
        rf_false_count+=1
    else:
        rf_true_count+=1

rf_accuracy = rf_false_count/n*100
print(f'Random Forest Accuracy: {rf_accuracy}%')

NameError: name 'rf_rand_y_pred' is not defined

In [95]:
lr_false_count = 0
lr_true_count = 0

for i in lr_rand_y_pred:
    if i == False:
        lr_false_count+=1
    else:
        lr_true_count+=1

lr_accuracy = lr_false_count/n*100
print(f'Logistic Regression Accuracy: {lr_accuracy}%')

Logisitc Regression Accuracy: 96.82%


In [96]:
svc_false_count = 0
svc_true_count = 0

for i in svc_rand_y_pred:
    if i == False:
        svc_false_count+=1
    else:
        svc_true_count+=1

svc_accuracy = svc_false_count/n*100
print(f'Linear SVC Accuracy: {svc_accuracy}%')

Linear SVC Accuracy: 97.76%
